In [106]:
# import the necessary libraries for the analyzis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [107]:
# read college player statistics from 2009 to 2022
# the data can be found in two different csv files, one contains stats from 2009 to 2021
# while the other one contains the latest statistics (2022)
college1 = pd.read_csv('Data\CollegeBasketballPlayers2009-2021.csv')
college2 = pd.read_csv('Data\CollegeBasketballPlayers2022.csv')

# the other data source contains draft picks at the nba for each year from 2009 to 2021
draft = pd.read_excel('Data\DraftedPlayers2009-2021.xlsx')

c:\Apps\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [108]:
# first of all, lets concatenate the college statistical dataframes
college = pd.concat([college1,college2])
print(college1.size+college2.size == college.size)

True


In [109]:
college.head()

,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,dgbpm,oreb,dreb,treb,ast,stl,blk,pts,Unnamed: 64,Unnamed: 65
0,DeAndrae Ross,South Alabama,SB,26,29.5,97.3,16.6,42.5,44.43,1.6,...,-1.941150,0.1923,0.6154,0.8077,1.1923,0.3462,0.0385,3.8846,NaN,6.22026
1,Pooh Williams,Utah St.,WAC,34,60.9,108.3,14.9,52.4,54.48,3.8,...,-0.247934,0.6765,1.2647,1.9412,1.8235,0.4118,0.2353,5.9412,NaN,3.94375
2,Jesus Verdejo,South Florida,BE,27,72.0,96.2,21.8,45.7,47.98,2.1,...,-0.883163,0.6296,2.3333,2.9630,1.9630,0.4815,0.0000,12.1852,NaN,10.92680
3,Mike Hornbuckle,Pepperdine,WCC,30,44.5,97.7,16.0,53.6,53.69,4.1,...,-0.393459,0.7000,1.4333,2.1333,1.1000,0.5667,0.1333,4.9333,NaN,6.77427
4,Anthony Brown,Pacific,BW,33,56.2,96.5,22.0,52.8,54.31,8.3,...,-0.668318,1.4242,3.3030,4.7273,0.8485,0.4545,0.3333,7.5758,NaN,0.00000


In [110]:
# since the draft data set has merged cells in the table header the first row must be dropped
draft.drop(0,axis=0,inplace=True)

In [111]:
# rename the ROUND.1 column to PICK, and modify the PLAYER to player_name 
# so it can be act as a key during the join with the college data set
draft.rename(
    columns={
        "PLAYER": "player_name", 
        "TEAM": "drafted_by", 
        "YEAR" : "draft_year", 
        "ROUND" : "draft_round", 
        "ROUND.1" : "draft_pick"},
        inplace=True)

# also lower all column names
draft.columns = draft.columns.str.lower()

In [112]:
draft.columns

Index(['player_name', 'drafted_by', 'affiliation', 'draft_year', 'draft_round',
       'draft_pick', 'overall'],
      dtype='object')

In [113]:
# join (merge) the college set with the draft data to identify those players who have been drafted after playing in college
df = pd.merge(college,draft,how='left',on='player_name')

In [114]:
# df.info()
df.columns

Index(['player_name', 'team', 'conf', 'GP', 'Min_per', 'Ortg', 'usg', 'eFG',
       'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM', 'FTA',
       'FT_per', 'twoPM', 'twoPA', 'twoP_per', 'TPM', 'TPA', 'TP_per',
       'blk_per', 'stl_per', 'ftr', 'yr', 'ht', 'num', 'porpag', 'adjoe',
       'pfr', 'year', 'pid', 'type', 'Rec Rank', 'ast/tov', 'rimmade',
       'rimmade+rimmiss', 'midmade', 'midmade+midmiss',
       'rimmade/(rimmade+rimmiss)', 'midmade/(midmade+midmiss)', 'dunksmade',
       'dunksmiss+dunksmade', 'dunksmade/(dunksmade+dunksmiss)', 'pick',
       'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm',
       'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk',
       'pts', 'Unnamed: 64', 'Unnamed: 65', 'drafted_by', 'affiliation',
       'draft_year', 'draft_round', 'draft_pick', 'overall'],
      dtype='object')

In [138]:
# create a new column to identify the drafted players
df['drafted_flag'] = (~df.overall.isnull())*1

In [139]:
df.drafted_flag.value_counts()

0    63568
1     1492
Name: drafted_flag, dtype: int64

In [143]:
df.player_name[df.drafted_flag == 1].value_counts()

Justin Jackson         22
Marcus Thornton        20
Cameron Johnson        14
Jalen Johnson          14
James Johnson          11
                       ..
DeMarre Carroll         1
Scottie Barnes          1
Dejounte Murray         1
Talen Horton-Tucker     1
Julius Randle           1
Name: player_name, Length: 595, dtype: int64

In [145]:
df[df.player_name == 'Justin Jackson']

,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,pts,Unnamed: 64,Unnamed: 65,drafted_by,affiliation,draft_year,draft_round,draft_pick,overall,drafted_flag
3570,Justin Jackson,Tennessee,SEC,1,0.1,0.0,0.0,0.0,0.00,0.0,...,0.0000,NaN,0.000000,Denver Nuggets,Maryland,2018.0,2,13,43,1
3571,Justin Jackson,Tennessee,SEC,1,0.1,0.0,0.0,0.0,0.00,0.0,...,0.0000,NaN,0.000000,Portland Trail Blazers,North Carolina,2017.0,1,15,15,1
11966,Justin Jackson,Morgan St.,MEAC,16,5.6,83.1,13.9,35.7,36.79,4.8,...,0.6875,Wing G,3.343690,Denver Nuggets,Maryland,2018.0,2,13,43,1
11967,Justin Jackson,Morgan St.,MEAC,16,5.6,83.1,13.9,35.7,36.79,4.8,...,0.6875,Wing G,3.343690,Portland Trail Blazers,North Carolina,2017.0,1,15,15,1
12674,Justin Jackson,Cincinnati,BE,35,32.2,84.6,15.8,53.4,49.62,7.8,...,2.5429,Wing F,0.414616,Denver Nuggets,Maryland,2018.0,2,13,43,1
12675,Justin Jackson,Cincinnati,BE,35,32.2,84.6,15.8,53.4,49.62,7.8,...,2.5429,Wing F,0.414616,Portland Trail Blazers,North Carolina,2017.0,1,15,15,1
15762,Justin Jackson,Cincinnati,BE,37,52.0,96.6,16.9,52.0,52.55,10.1,...,5.0811,C,0.556863,Denver Nuggets,Maryland,2018.0,2,13,43,1
15763,Justin Jackson,Cincinnati,BE,37,52.0,96.6,16.9,52.0,52.55,10.1,...,5.0811,C,0.556863,Portland Trail Blazers,North Carolina,2017.0,1,15,15,1
19446,Justin Jackson,Cincinnati,BE,30,41.0,82.5,18.2,42.3,44.09,9.5,...,3.8333,C,1.197780,Denver Nuggets,Maryland,2018.0,2,13,43,1
19447,Justin Jackson,Cincinnati,BE,30,41.0,82.5,18.2,42.3,44.09,9.5,...,3.8333,C,1.197780,Portland Trail Blazers,North Carolina,2017.0,1,15,15,1
